# Σε αυτό το Notebook πραγματοποιείται η εκπαίδευση του μοντέλου


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle
from sklearn.utils import shuffle
from tensorflow.keras.layers import Input, Dense,add
from tensorflow.keras.models import Model
from datetime import timedelta
from tqdm import tqdm

import geopy.geocoders
from geopy.geocoders import Nominatim
import geopy.distance

Project_Path='/Users/nickkarras/PycharmProjects/ParkingViolationPredictionGraph_Git'

In [78]:
Distance_Data=pd.read_csv(Project_Path+ '/Data/Distance.csv',sep=',',index_col=0)
Final_Weather_Data=pd.read_csv(Project_Path+ '/Data/Final_Weather_Data.csv',low_memory=False,sep=',',index_col=0)
Legal_illegal=pd.read_csv(Project_Path+ '/Data/Scan_Data_Reg_2.3.csv',sep=',',index_col=0)
train_data=pd.read_csv(Project_Path+ '/Data/Train_TimeSeries.csv',sep=',',index_col=0)
test_data=pd.read_csv(Project_Path+ '/Data/Test_TimeSeries.csv',sep=',',index_col=0)
Names=['Slot_id','Slot_Timeint','Ilegality_Rate']

# Smoothing

Εδώ κάνω την παραπάνω διαδικασία εφαρμόζοντας smoothing και στο train-set και στο validation/test-set

.

H παρακάτω συνάρτηση δέχεται  ένα σύνολο δεδομένων και επιστρέφει ένα smoothed-σύνολο δεδομένων. Ουσιαστικά χρησιμοποιώντας ένα data sample δημιουργεί τρία, δηλαδή τριπλασιάζει το σύνολο δεδομένων. Έχω περιγράψει αναλυτικά την διαδικασία στην αναφορά.

# Temporal Smoothing

In [70]:
def Return_Mean(Time_Slot,Sector,train_data,Names):
    Help=train_data[Names]
    ILegal_Sum = Help.groupby(['Slot_id','Slot_Timeint']).Ilegality_Rate.mean().reset_index()
    ILegal_Sum=ILegal_Sum.loc[(ILegal_Sum['Slot_id']  == str(Sector)) & (ILegal_Sum['Slot_Timeint'] ==str(Time_Slot))]
    a=ILegal_Sum['Ilegality_Rate'].values.tolist()
    if ILegal_Sum.empty:
        mean=0.4
    else: 
        mean=a[0]
        
    if mean==0:
        mean=0.00001
    return mean

In [40]:
def Smouth(Legal_illegal,Names):
    #Παίρνει τα καταγεγραμμένα time-slots και τα μετατρέπει ώστε να αποτυπώνονται με το κέντρο τους "πχ: 7:00->7:30" 
    Time_Slots=[21600,25200,28800,32400,36000,39600,43200,46800,50400,54000,57600,61200,64800,68400,72000]
    Time_SlotsCenter=[]
    for i in range (0,len(Time_Slots)):
        Time_SlotsCenter.append((Time_Slots[i]+1800)/timedelta(days=1).total_seconds())
    Time_Slots=Time_SlotsCenter

    
    Scan_List2=Legal_illegal.values.tolist()
    NewData=[]
    Slots=[]
    for i in tqdm(range(0,len(Scan_List2))):
        Helper=[]
        Helper2=[]
        Rate=Scan_List2[i][6] #Πραγματικό ποστό παραβατικότητας στάθμευσης 
        Real_Time=Scan_List2[i][5] #Πραγματική ώρα του ελέγχου 

        #Μετράει για τον κάθε έλεγχο τις αποστάσεις απο όλα τα time slots 
        #Μετρώντας την απόσταση απο την πραγατική ώρα του ελέγχου με τα κέντρα των  timeslots
        #Βρίσκει για τον εκάστοτε έλεγχο ποια είναι τα τρία κοντινότερα time-slots 
        Distances=[]
        for j in range (0,len(Time_Slots)):
            Distances.append(abs(Time_Slots[j]-Real_Time))
        Slots=np.column_stack((Time_Slots, Distances))
        Slots = sorted(Slots, key=lambda x: x[1])
        Slot1,Slot2,Slot3=Slots[0][0],Slots[1][0],Slots[2][0] #3 κοντινότερα  time-slots
        D1,D2,D3=Slots[0][1],Slots[1][1],Slots[2][1] #Χρονικές Αποστάσεις απο τα 3 κοντινότερα  time-slots
        Sector=Scan_List2[i][0]
       
        #Δημιουργέι ένα data sample με όλα τα χαρακτηριστικά από το αρχικό βάζοντας σαν time slot το κοντινότερο "Slot1"
        #Βάζει σαν ποσοστό παραβατικότητας το πραγματικό 'Rate'
        #Σαν απόσταση 0, γιατί αφού βάζουμε το πραγματικό ποσοστό είναι σαν να θεωρούμε ότι βρίσκεται ακριβώς στο 
        #κέντρο του time-slot. Εξηγώ τον τύπο αναλυτικότερα στη αναφορά.
        Mean=Return_Mean(Slot1,Sector,Legal_illegal,Names)
        Diff=Rate/Mean
        
        Helper=Scan_List2[i][:11]
        Helper.append(Slot1)
        Helper.append(Rate)
        Helper.append(0)
        NewData.append(Helper)
        
        #Δημιουργέι ένα δέυτερο data sample με όλα τα χαρακτηριστικά από το αρχικό βάζοντας 
        #σαν time slot το δέυτερο  κοντινότερο
        #Βάζει σαν ποσοστό παραβατικότητας αυτό που προκύπτει εφαρμόζοντας την γκαουσιανή
        #Βάζει την απόσταση 'D2' απο το δέυτερο κοντινότερο timeslot
        
        Mean=Return_Mean(Slot2,Sector,Legal_illegal,Names)
        Applied_Mean=Mean*Diff
        if Applied_Mean>1:
            Applied_Mean=1
        
        
        Helper=Scan_List2[i][:11]
        Helper.append(Slot2)
        #Ο τύπος είναι "-απόσταση σε λεπτά/210 λεπτά"
        #Το '0.14583' είναι το 210 λεπτά, σύμφωνα με την κανονικοποήση που έχω κάνει στην ώρα
        X1=((-D2)/0.14583) 
        X2=np.exp(X1)

        X3=(1-X2)*Applied_Mean
        r=(Rate*X2)+X3
    
        Helper.append(r)
        Helper.append(D2)
        NewData.append(Helper)

        #Δημιουργέι ένα τρίτο data sample με όλα τα χαρακτηριστικά από το αρχικό βάζοντας σαν time slot 
        #το τρίτο κοντινότερο
        #Βάζει σαν ποσοστό παραβατικότητας αυτο που προκύπτει εφαρμόζοντας την γκαουσιανή
        #Βάζει την απόσταση 'D3' απο το τρίτο κοντινότερο timeslot
        
        Mean=Return_Mean(Slot3,Sector,Legal_illegal,Names)
        Applied_Mean=Mean*Diff
        if Applied_Mean>1:
            Applied_Mean=1
        
        Helper=Scan_List2[i][:11]
        Helper.append(Slot3)
        X1=((-D3)/0.14583)
        X2=np.exp(X1)
  
        X3=(1-X2)*Applied_Mean
        r=(Rate*X2)+X3
        
        Helper.append(r)
        Helper.append(D3)
        NewData.append(Helper)

    Col=['Slot_id','Key','Date_Sin','Slot_Timeint','Covid','Time_Int','Ilegality_Rate','Holidays','Capacity','Week_Day_Sin','Month_Sin','Real_Time','Real_Rate','Time_Distance']
    Legal_illegal = pd.DataFrame (NewData, columns = Col)
    
    
    #Δημιουργεί το τελικό dataset διαγράφοντας τα χαρακτηριστικά που δέν χρειάζονται. 
    #Αλλάζει θέση σε ένα χαρακτηριστικό ώστε να είναι στην σωστή. 
    Legal_illegal=Legal_illegal.drop(['Slot_Timeint'], axis=1)
    Legal_illegal=Legal_illegal.drop(['Time_Int'], axis=1)
    Legal_illegal=Legal_illegal.drop(['Ilegality_Rate'], axis=1)
    a=Legal_illegal['Time_Distance']
    Legal_illegal=Legal_illegal.drop(['Time_Distance'], axis=1)
    Legal_illegal.insert(8, "Time_Distance", a, True)
    return Legal_illegal

Εφόσον εφόσον χρησιμοποιήσαμε πάνω ένα sample για να δημιουργήσουμε άλλα 2 και δώσαμε τα ίδια χαρακτηριστικά έκτος του ποσοστού παραβατικότητας και του timeslot. Έχουμε δώσει και ίδο 'Key' το key είναι datetime και το χρησιμοποιώ στην συνεχεία για να κάνω merge τις ωριαίες τιμές του καιρού. Εφόσον τα 2 νέα δείγματα αντιπροσωπεύουν άλλη ώρα πρέπει να αλλάξει και το 'Key'. Στην παρακάτω συνάρτηση βρίσκουμε τα σωστά  key.

In [41]:
def Get_Weather(Legal_illegal):
    #Μεταρέπει την ώρα σε κανονική μορφή 
    Time=Legal_illegal['Real_Time']*timedelta(days=1).total_seconds()
    Time=Time/3600
    Time=Time.astype(int)
    Time=Time.values.tolist()
    NewT=[]
    #Αν είναι η ώρα 9:00 την μετατρέπει σε 09:00 για να έχει την ιδια μορφή με το key του καιρού
    for i in range (0,len(Time)):
        Str=str(Time[i])
        if Time[i]>=10:
            NewT.append(Str)
        else:
            NewT.append('0'+Str)

    #Βάζει στο τέλος της κάθε ώρας το ":00" για να έχει την ιδια μορφή με το key του καιρού
    Time=pd.DataFrame(NewT,columns=["Hour"])
    Time= Time["Hour"].map(str)+ ':00'
    #Παίρνει και την ημερομηνία
    T_List=Legal_illegal.values.tolist()
    Date=[]
    for i in range (0,len(Legal_illegal)):
        D,H=T_List[i][1].split(' ')
        Date.append(D)

    #Βάζει σαν νεο key την ημερομηνία
    Legal_illegal=Legal_illegal.drop(['Key'], axis=1)
    Legal_illegal.insert(1, "Key", Date, True)
    
    #Προσθέτει στο νέο key την ώρα
    Key_Weather=Legal_illegal['Key'].map(str) + ' ' + Time
    Legal_illegal=Legal_illegal.drop(['Key'], axis=1)
    
    #Πλεον έχω το σωστό key
    Legal_illegal.insert(1, "Key", Key_Weather, True)
    
    #Κάνω merge με τα δεδομένα του καιρού
    Legal_illegal=pd.merge(Legal_illegal, Final_Weather_Data, on='Key')
    
    #Legal_illegal=Legal_illegal.drop(['Key'], axis=1)
    #print(Legal_illegal.columns)
    return Legal_illegal


In [42]:
def GetMeans(Scans_Data2):
    Scans_Data2['Key1'] = Scans_Data2['Slot_id'].map(str)+ '+' +Scans_Data2['Key'].map(str)+ '+' +Scans_Data2['Date_Sin'].map(str) + '+' + Scans_Data2['Covid'].map(str)+ '+' + Scans_Data2['Holidays'].map(str)+ '+'+Scans_Data2['Capacity'].map(str)+ '+'+Scans_Data2['Week_Day_Sin'].map(str)+ '+'+Scans_Data2['Month_Sin'].map(str)+ '+'+Scans_Data2['Real_Time'].map(str)+ '+'+Scans_Data2['temp'].map(str)+ '+'+Scans_Data2['humidity'].map(str)
    Scans_Data2=Scans_Data2['Key1'],Scans_Data2['Time_Distance'],Scans_Data2['Real_Rate']
    Headers=['Key1','Time_Distance','Real_Rate']
    Scans_Data2 = pd.concat(Scans_Data2, axis=1, keys=Headers)
    
    Time_Int = Scans_Data2.groupby(["Key1"]).Time_Distance.mean().reset_index()
    ILegal_Sum = Scans_Data2.groupby(["Key1"]).Real_Rate.mean().reset_index()
    Legal_Sum=ILegal_Sum['Key1'],Time_Int['Time_Distance'],ILegal_Sum['Real_Rate']
    Headers=['Key1','Time_Distance','Real_Rate']
    Legal_illegal = pd.concat(Legal_Sum, axis=1, keys=Headers)
    
    
    Scan_List2=Legal_illegal.values.tolist()
    Slot_id=[]
    Key=[]
    Date_Sin=[]
    Covid=[]
    Holidays=[]
    Capacity=[]
    Week_Day_Sin=[]
    Month_Sin=[]
    Real_Time=[]
    temp=[]
    humidity=[]
    
    for i in tqdm(range(0,len(Scan_List2))):
        Slot_id_V,Key_V,Date_Sin_V,Covid_V,Holidays_V,Capacity_V,Week_Day_Sin_V,Month_Sin_V,Real_Time_V,temp_V,humidity_V=Scan_List2[i][0].split('+')
        Slot_id.append(Slot_id_V)
        Key.append(Key_V)
        Date_Sin.append(Date_Sin_V)
        Covid.append(Covid_V)
        Holidays.append(Holidays_V)
        Capacity.append(Capacity_V)
        Week_Day_Sin.append(Week_Day_Sin_V)
        Month_Sin.append(Month_Sin_V)
        Real_Time.append(Real_Time_V)
        temp.append(temp_V)
        humidity.append(humidity_V)
    Legal_illegal.insert(1, "Slot_id", Slot_id, True)
    Legal_illegal.insert(2, "Key", Key, True)
    Legal_illegal.insert(3, "Date_Sin", Date_Sin, True)
    Legal_illegal.insert(4, "Covid",  Covid, True)
    Legal_illegal.insert(5, "Holidays", Holidays, True)
    Legal_illegal.insert(6, "Capacity", Capacity, True)
    Legal_illegal.insert(7, "Week_Day_Sin", Week_Day_Sin, True)
    Legal_illegal.insert(8, "Month_Sin", Month_Sin, True)
    Legal_illegal.insert(9, "Real_Time", Real_Time, True)
    Legal_illegal.insert(10, "temp", temp, True)
    Legal_illegal.insert(11, "humidity", humidity, True)
    Legal_illegal=Legal_illegal.drop(['Key1'], axis=1)
    
    Columns2=['Slot_id','Key', 'Date_Sin', 'Covid', 'Holidays', 'Capacity', 'Week_Day_Sin',
    'Month_Sin', 'Time_Distance', 'Real_Time', 'Real_Rate', 'temp','humidity']
    Legal_illegal=Legal_illegal[Columns2]
    return Legal_illegal

Η παρακάτω συνάρτηση κάνει merge με το αρχείο από τις αποστάσεις των τομέων από τα 19 σημεία ενδιαφέροντος και διαγράφει το slot id γιατί δεν το χρησιμοποιούμε στην εκπαίδευση 

In [43]:
def Get_Slot_Distances(Legal_illegal):
    Legal_illegal['Slot_id'] = Legal_illegal['Slot_id'].astype(float)
    Legal_illegal['Slot_id'] = Legal_illegal['Slot_id'].astype(int)
    Legal_illegal=pd.merge(Legal_illegal, Distance_Data, on='Slot_id')
    return Legal_illegal

Η παρακάτω συνάρτηση ουσιαστικά ενώνει τις τρεις παραπάνω συναρτήσεις και αυτές αποτελούν την διαδικασία του smoothing. Όπου έχοντας σαν είσοδο το data-set που δημιούργησα στο προηγούμενο notebook παίρνω στην έξοδο το smoothed dataset

In [44]:
def Apply_Smoothing(Legal_illegal,Names):
    Legal_illegal=Smouth(Legal_illegal,Names)
    Legal_illegal=Get_Weather(Legal_illegal)
    Legal_illegal=GetMeans(Legal_illegal)
    Legal_illegal=Get_Slot_Distances(Legal_illegal)
    return Legal_illegal

In [45]:
def Prepare_Test(Test): 
    #Σαν απόσταση βάζει 0, γιατί αφού έχουμε το πραγματικό ποσοστό είναι σαν να θεωρούμε ότι βρίσκεται ακριβώς στο 
    #κέντρο του time-slot. Εξηγώ τον τύπο αναλυτικότερα στη αναφορά.
    Test['Time_Distance']=0
    Test=Test.drop(['Time_Int'], axis=1)
    a=Test['Slot_Timeint']
    b=Test['Ilegality_Rate']
    Test=Test.drop(['Slot_Timeint'], axis=1)
    Test=Test.drop(['Ilegality_Rate'], axis=1)
    Test.insert(9, "Real_Time", a, True)
    Test.insert(10, "Real_Rate", b, True)
    Test=pd.merge(Test, Final_Weather_Data, on='Key')
    Test=pd.merge(Test, Distance_Data, on='Slot_id')
    return Test

In [46]:
TestDF=Prepare_Test(test_data) #No smoothing
TrainDF=Apply_Smoothing(train_data,Names) #Smoothing
TrainDF=TrainDF.drop(['Time_Distance'], axis=1)
TestDF=TestDF.drop(['Time_Distance'], axis=1)
TrainDF.to_csv(Project_Path+ '/Data/Ts_Train_TimeSeries.csv')
TestDF.to_csv(Project_Path+ '/Data/Ts_Test_TimeSeries.csv')

  7%|▋         | 6214/88870 [00:55<12:20, 111.58it/s]


KeyboardInterrupt: 

In [ ]:
TrainDF=TrainDF.drop(['Slot_id'], axis=1)
TrainDF=TrainDF.drop(['Key'], axis=1)
TestDF=TestDF.drop(['Slot_id'], axis=1)
TestDF=TestDF.drop(['Key'], axis=1)
train_data=TrainDF.drop(['Real_Rate'], axis=1)
test_data=TestDF.drop(['Real_Rate'], axis=1)

Standar_Scaller = StandardScaler()
train_data=Standar_Scaller.fit_transform(train_data)
with open(Project_Path + '/Standar_Scaller.pkl', 'wb') as f:
    pickle.dump(Standar_Scaller, f,  protocol=2)